## 第5章　遗传算法

### 5.2　通用的遗传算法

In [3]:
# chromosome.py
from __future__ import annotations
from typing import TypeVar, Tuple, Type
from abc import ABC, abstractmethod

T = TypeVar('T', bound='Chromosome') # for returning self

# Base class for all chromosomes; all methods must be overridden
class Chromosome(ABC):
    @abstractmethod
    def fitness(self) -> float:
        ...
    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...
    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...
    @abstractmethod
    def mutate(self) -> None:
        ...

In [10]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Tuple, Callable
from enum import Enum
from random import choices, random
from heapq import nlargest
from statistics import mean

C = TypeVar('C', bound=Chromosome) # type of the chromosomes

class GeneticAlgorithm(Generic[C]):
    SelectionType = Enum("SelectionType", "ROULETTE TOURNAMENT")
    def __init__(self, initial_population: List[C], threshold: float, max_generations:\
     int = 100, mutation_chance: float = 0.01, crossover_chance: float = 0.7, \
     selection_type: SelectionType = SelectionType.TOURNAMENT) -> None: 
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
        self._fitness_key: Callable = type(self._population[0]).fitness
    
    # Use the probability distribution wheel to pick 2 parents
    # Note: will not work with negative fitness results
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        return tuple(choices(self._population, weights=wheel, k=2))
    
    # Choose num_participants at random and take the best 2
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        participants: List[C] = choices(self._population, k=num_participants)
        return tuple(nlargest(2, participants, key=self._fitness_key))
    
    # Replace the population with a new generation of individuals
    def _reproduce_and_replace(self) -> None:
        new_population: List[C] = []
        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            # pick the 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                parents: Tuple[C, C] = self._pick_roulette([x.fitness() for x in  
                self._population])
            else:
                parents = self._pick_tournament(len(self._population) // 2)
            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                new_population.extend(parents)
        # if we had an odd number, we'll have 1 extra, so we remove it
        if len(new_population) > len(self._population):
            new_population.pop()
        self._population = new_population # replace reference
        
    # With _mutation_chance probability mutate each individual
    def _mutate(self) -> None:
        for individual in self._population:
            if random() < self._mutation_chance:
                individual.mutate()
    # Run the genetic algorithm for max_generations iterations
    # and return the best individual found
    def run(self) -> C:
        best: C = max(self._population, key=self._fitness_key)
        for generation in range(self._max_generations):
            # early exit if we beat threshold
            if best.fitness() >= self._threshold: 
                return best
            print(f"Generation {generation} Best {best.fitness()} Avg " +\
                  f"{mean(map(self._fitness_key, self._population))}")
            self._reproduce_and_replace()
            self._mutate()
            highest: C = max(self._population, key=self._fitness_key)
            if highest.fitness() >best.fitness():
                best = highest # found a new best
        return best # best we found in _max_generations

In [18]:
from __future__ import annotations
from typing import Tuple, List
from random import randrange, random
from copy import deepcopy

class SimpleEquation(Chromosome):
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y

    def fitness(self) -> float: # 6x - x^2 + 4y - y^2
        return 6 * self.x - self.x * self.x + 4 * self.y - self.y * self.y

    @classmethod
    def random_instance(cls) -> SimpleEquation:
        return SimpleEquation(randrange(100), randrange(100))

    def crossover(self, other:SimpleEquation) ->Tuple[SimpleEquation,SimpleEquation]:
        child1:SimpleEquation = deepcopy(self)
        child2:SimpleEquation = deepcopy(other)
        child1.y = other.y
        child2.y = self.y
        return child1, child2

    def mutate(self) -> None:
        if random() > 0.5: # mutate x
            if random() > 0.5:
                self.x += 1
            else:
                self.x -= 1
        else: # otherwise mutate y
            if random() > 0.5:
                self.y += 1
            else:
                self.y -= 1

    def __str__(self) -> str:
        return f"X: {self.x} Y: {self.y} Fitness: {self.fitness()}"
    
initial_population: List[SimpleEquation] = [SimpleEquation.random_instance() for _ in range(20)]
ga: GeneticAlgorithm[SimpleEquation] = GeneticAlgorithm(initial_population=initial_population, \
    threshold=13.0, max_generations = 100, mutation_chance = 0.1, crossover_chance = 0.7)
result:SimpleEquation = ga.run()
print(result)

Generation 0 Best -4 Avg -5341.3
Generation 1 Best -4 Avg -632.15
Generation 2 Best 3 Avg -84.05
Generation 3 Best 3 Avg -4.4
Generation 4 Best 3 Avg 2
Generation 5 Best 3 Avg 3
Generation 6 Best 4 Avg 3.15
Generation 7 Best 4 Avg 3.15
Generation 8 Best 4 Avg 3.6
Generation 9 Best 4 Avg 4
Generation 10 Best 4 Avg 4
Generation 11 Best 4 Avg 3.85
Generation 12 Best 4 Avg 3.45
Generation 13 Best 4 Avg 4
Generation 14 Best 4 Avg 4
Generation 15 Best 4 Avg 4
Generation 16 Best 4 Avg 3.9
Generation 17 Best 9 Avg 4.2
Generation 18 Best 9 Avg 5.2
Generation 19 Best 9 Avg 8.15
Generation 20 Best 12 Avg 9.15
Generation 21 Best 12 Avg 9.5
Generation 22 Best 12 Avg 11.2
X: 3 Y: 2 Fitness: 13


### 5.4　重新考虑SEND+MORE=MONEY问题

In [20]:
from __future__ import annotations
from typing import Tuple, List
from random import shuffle, sample
from copy import deepcopy

class SendMoreMoney2(Chromosome):
    def __init__(self, letters: List[str]) -> None:
        self.letters: List[str] = letters

    def fitness(self) -> float:
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        return 1 / (difference + 1)

    @classmethod
    def random_instance(cls) ->SendMoreMoney2:
        letters = ["S", "E", "N", "D", "M", "O", "R", "Y", " ", " "]
        shuffle(letters)
        return SendMoreMoney2(letters)

    def crossover(self, other: SendMoreMoney2) -> Tuple[SendMoreMoney2, SendMoreMoney2]:
        child1: SendMoreMoney2 = deepcopy(self)
        child2: SendMoreMoney2 = deepcopy(other)
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        l1, l2 = child1.letters[idx1], child2.letters[idx2]
        child1.letters[child1.letters.index(l2)], child1.letters[idx2] = child1.\
         letters[idx2], l2
        child2.letters[child2.letters.index(l1)], child2.letters[idx1] = child2.\
         letters[idx1], l1
        return child1, child2

    def mutate(self) -> None: # swap two letters' locations
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        self.letters[idx1], self.letters[idx2] = self.letters[idx2], self.letters[idx1]

    def __str__(self) -> str:
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        return f"{send} + {more} = {money} Difference: {difference}"

In [24]:
initial_population: List[SendMoreMoney2] = [SendMoreMoney2.random_instance() for _ in range(1000)]
ga: GeneticAlgorithm[SendMoreMoney2] = GeneticAlgorithm(initial_population=initial_population,\
     threshold=1.0, max_generations = 1000, mutation_chance = 0.2, crossover_chance = 0.7,\
     selection_type=GeneticAlgorithm.SelectionType.ROULETTE)
result: SendMoreMoney2 = ga.run()
print(result)

Generation 0 Best 0.015151515151515152 Avg 0.00012380157887518917
Generation 1 Best 0.05555555555555555 Avg 0.001446811133004254
Generation 2 Best 0.5 Avg 0.006892118852242911
Generation 3 Best 0.5 Avg 0.022110458917417124
Generation 4 Best 0.5 Avg 0.10643477040279649
7539 + 815 = 8354 Difference: 0


### 5.5　优化列表压缩算法

In [27]:
from __future__ import annotations
from typing import Tuple, List, Any
from random import shuffle, sample
from copy import deepcopy
from zlib import compress
from sys import getsizeof
from pickle import dumps

# 165 bytes compressed
PEOPLE: List[str] = ["Michael", "Sarah", "Joshua", "Narine", "David", "Sajid", "Melanie",
     "Daniel", "Wei", "Dean", "Brian", "Murat", "Lisa"] 

class ListCompression(Chromosome):
    def __init__(self, lst: List[Any]) -> None:
        self.lst: List[Any] = lst

    @property
    def bytes_compressed(self) -> int:
        return getsizeof(compress(dumps(self.lst)))

    def fitness(self) -> float:
        return 1 / self.bytes_compressed

    @classmethod
    def random_instance(cls) -> ListCompression:
        mylst: List[str] = deepcopy(PEOPLE)
        shuffle(mylst)
        return ListCompression(mylst)

    def crossover(self, other: ListCompression) -> Tuple[ListCompression, ListCompression]:
        child1: ListCompression = deepcopy(self)
        child2: ListCompression = deepcopy(other)
        idx1, idx2 = sample(range(len(self.lst)), k=2)
        l1, l2 = child1.lst[idx1], child2.lst[idx2]
        child1.lst[child1.lst.index(l2)], child1.lst[idx2] = child1.lst[idx2], l2
        child2.lst[child2.lst.index(l1)], child2.lst[idx1] = child2.lst[idx1], l1
        return child1, child2

    def mutate(self) -> None: # swap two locations
        idx1, idx2 = sample(range(len(self.lst)), k=2)
        self.lst[idx1], self.lst[idx2] = self.lst[idx2], self.lst[idx1]

    def __str__(self) -> str:
        return f"Order: {self.lst} Bytes: {self.bytes_compressed}"


initial_population: List[ListCompression] = [ListCompression.random_instance()\
 for _ in range(1000)]
ga: GeneticAlgorithm[ListCompression] = GeneticAlgorithm(initial_population=\
 initial_population, threshold=1.0, max_generations = 10, mutation_chance =\
 0.2, crossover_chance = 0.7, selection_type=GeneticAlgorithm.SelectionType.TOURNAMENT)
result: ListCompression = ga.run()
print(result)

Generation 0 Best 0.007246376811594203 Avg 0.00698954663440981
Generation 1 Best 0.007246376811594203 Avg 0.007110303035033327
Generation 2 Best 0.007246376811594203 Avg 0.0071665221324269494
Generation 3 Best 0.0072992700729927005 Avg 0.007194049221621377
Generation 4 Best 0.0072992700729927005 Avg 0.007191819750443402
Generation 5 Best 0.0072992700729927005 Avg 0.007252530306648797
Generation 6 Best 0.0072992700729927005 Avg 0.007265131944288714
Generation 7 Best 0.0072992700729927005 Avg 0.007271300443416216
Generation 8 Best 0.0072992700729927005 Avg 0.0072698551288110595
Generation 9 Best 0.0072992700729927005 Avg 0.007269286930808161
Order: ['Lisa', 'Melanie', 'Daniel', 'Narine', 'Joshua', 'Michael', 'Sarah', 'David', 'Sajid', 'Murat', 'Dean', 'Brian', 'Wei'] Bytes: 137
